<a href="https://colab.research.google.com/github/mjag7682/CS9-1-NLP-for-Twitter-Data-for-predicting-stocks/blob/master/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GPU connect

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

## Imports and Installs


In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1-1-NxTeGh9HktAvcYMDLuCe0jJK6Cngb'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Combined_FAANG_percentage_2.2.csv')

id = '1-4wGVlhCObAoAM_DOLL3D4YsJhOb1ZSj'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Combined_FAANG_binary_previous.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install contractions
!pip install emoji
!pip install ekphrasis

import pandas as pd
import re
import emoji
import nltk
import contractions
import torch
from sklearn.model_selection import train_test_split

nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import TweetTokenizer 
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize 
from nltk.tokenize.treebank import TreebankWordDetokenizer
import requests 
from pprint import pprint
import numpy as np
from nltk.stem import PorterStemmer

import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from tensorflow import keras

import os
import pprint
import json
import random
import string
import sys
from ekphrasis.classes.segmenter import Segmenter
import itertools

## Load data

In [ ]:
df_comb = pd.read_csv('/content/Combined_FAANG_percentage_2.2.csv', sep=',')
df_comb = df_comb[df_comb['message'].notna()]
df = df_comb.sample(frac=1) # consider whole data and shuffle it
df

Drop before a particular date depending on duration of data considered


In [ ]:
df.drop(df[df['Date'] <= '2019-07-20'].index, inplace = True) 
df

In [ ]:
df["label"].value_counts()

Drop neutrals and replace negative label -1 with 0. 


In [ ]:
df.drop(df[df['label'] == 0].index, inplace = True) 
df["label"].replace({-1: 0}, inplace=True) # therefore, 0 - negative, 1 - positive
df

In [ ]:
df["label"].value_counts()

## Split data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.1, random_state=42)

## Implement XGBoost

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
from xgboost import XGBClassifier

# Pipeline to run XGBoost with 2 vectorizations techniques - CountVectorizer and TfIdfTransformer
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', XGBClassifier())])

# Paramters to be tested with for CountVectorizer, TfidfTransformer and XGBClassifier
tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 0.1,0.5]
}


Train, Predict and print Classification Report

In [ ]:
from sklearn.metrics import classification_report
score = 'f1_macro'
clf = GridSearchCV(text_clf, tuned_parameters, cv=5, scoring=score)

# Train the model
clf.fit(x_train, y_train)

# Predict and print classification report
print(classification_report(y_test, clf.predict(x_test), digits=4))

Display parameters through which model gave best results


In [ ]:
clf.best_params_

Display estimator that gave the best results


In [ ]:
clf.best_estimator_

## Save the model

In [ ]:
import pickle

with open('/content/drive/My Drive/xgboost.bin', 'wb') as f:

    pickle.dump(clf, f)